## Mersenne Field 
```python
def sqrt(self):
    assert self.modulus % 4 == 3
    return self ** ((self.modulus + 1) // 4)
```

By [Euler's criterion](https://en.wikipedia.org/wiki/Euler%27s_criterion), if there is an $x$ s.t. $x^2 \equiv a \pmod p$, then $a^{\frac{p-1}{2}}\equiv 1 \pmod p$.

So $x^2 \equiv a^{\frac{p-1}{2}} a \equiv a^{\frac{p+1}{2}} \pmod p$, also by $p \equiv 3 \pmod 4$, $\frac{p+1}{4}$ is an integer. 

Therefore $x \equiv a^{\frac{p+1}{4}} \pmod p$

The `mersenne_31.rs` file in Plonky3 provided Rust code defines the `Mersenne31` prime field using the Mersenne prime $2^{31} - 1$. This field is used in cryptographic and mathematical computations due to its efficient arithmetic properties. The code includes implementations for various traits for field operations such as `Add`, `Sub`, `Mul`, `Div` to enable field arithmetic operations of `Mersenne31` values. Additionally, it provides methods for creating new `Mersenne31` instances, checking equality, hashing, and performing field-specific operations like exponentiation and inversion. Here is a simplified implementation using Python with detailed annotation:

In [3]:
# Base class for elements in a finite field
class FieldElement():
    def __init__(self, value):
        # If the input is an instance of FieldElement, extract its value
        if isinstance(value, self.__class__):
            value = value.value
        # Store the value modulo the field's modulus
        self.value = value % self.modulus

    # Addition operation
    def __add__(self, other):
        # Determine the value to add based on the type of 'other'
        othervalue = other if isinstance(other, int) else other.value
        # Return a new instance with the sum modulo the modulus
        return self.__class__((self.value + othervalue) % self.modulus)

    # Subtraction operation
    def __sub__(self, other):
        # Determine the value to subtract based on the type of 'other'
        othervalue = other if isinstance(other, int) else other.value
        # Return a new instance with the difference modulo the modulus
        return self.__class__((self.value - othervalue) % self.modulus)

    # Negation operation (additive inverse)
    def __neg__(self):
        # Return a new instance representing the negation
        return self.__class__(self.modulus - (self.value or self.modulus))

    # Multiplication operation
    def __mul__(self, other):
        # Determine the value to multiply based on the type of 'other'
        othervalue = other if isinstance(other, int) else other.value
        # Return a new instance with the product modulo the modulus
        return self.__class__((self.value * othervalue) % self.modulus)

    # Right-side addition (for operations like int + FieldElement)
    __radd__ = __add__
    # Right-side multiplication (for operations like int * FieldElement)
    __rmul__ = __mul__

    # Exponentiation operation
    def __pow__(self, other):
        # Return a new instance with the value raised to the power 'other' modulo the modulus
        return self.__class__(pow(self.value, other, self.modulus))

    # Multiplicative inverse
    def inv(self):
        # Compute the modular inverse using Python's built-in pow function with a negative exponent
        return self.__class__(
            pow(self.value, -1, self.modulus) if self.value else 0
        )

    # Square root operation (valid when modulus % 4 == 3)
    def sqrt(self):
        # Ensure the modulus is of the form 4k + 3
        assert self.modulus % 4 == 3
        # Return the square root using the exponentiation method specific to such moduli
        return self ** ((self.modulus + 1) // 4)

    # Division operation
    def __truediv__(self, other):
        # Convert 'other' to a FieldElement if it's an integer
        if isinstance(other, int):
            other = self.__class__(other)
        # Multiply by the multiplicative inverse of 'other'
        return self * other.inv()

    # Equality check
    def __eq__(self, other):
        # Determine the value to compare based on the type of 'other'
        othervalue = other if isinstance(other, int) else other.value
        # Check if the values are equal
        return self.value == othervalue

    # String representation for printing
    def __repr__(self):
        return '<' + str(self.value) + '>'

    # Convert the value to bytes (little-endian, 4 bytes)
    def to_bytes(self):
        return self.value.to_bytes(4, 'little')

    # Create an instance from bytes (little-endian)
    @classmethod
    def from_bytes(cls, bytez):
        return cls(int.from_bytes(bytez, 'little'))

# Subclass representing a specific finite field with modulus 7
class BabyMersenneElement(FieldElement):
    # Define the modulus as 2^3 - 1 = 7
    modulus = 2**3 - 1

# Alias for easier usage
BB = BabyMersenneElement


In [2]:
a = BB(3)
b = BB(5)

# Addition
c = a + b  # Result: <1> because (3 + 5) % 7 == 1
c

<1>

In [3]:
# Multiplication
d = a * b  # Result: <1> because (3 * 5) % 7 == 1
d

<1>

In [4]:
# Inverse
e = a.inv()  # Result: <5> because 3 * 5 % 7 == 1
e

<5>

In [5]:
# Division
f = a / b  # Result: <5> because 3 * 5^(-1) % 7 == 5
f

<2>

## Extension Field

In [4]:
class ExtendedFieldElement():
    def __init__(self, value):
        self.value = self._to_list(value)
        self.modulus = self.value[0].modulus

    def _to_list(self, value):
        if isinstance(value, self.__class__):
            return value.value
        elif isinstance(value, self.subclass):
            return [value] + [self.subclass(0)]*3
        elif isinstance(value, list):
            return [self.subclass(v) for v in value]
        elif isinstance(value, int):
            return [self.subclass(value)] + [self.subclass(0)]*3
        else:
            raise Exception("Incompatible value: {}".format(value))

    def __add__(self, other):
        othervalue = self._to_list(other)
        return self.__class__([x+y for x,y in zip(self.value, othervalue)])

    def __sub__(self, other):
        othervalue = self._to_list(other)
        return self.__class__([x-y for x,y in zip(self.value, othervalue)])

    def __mul__(self, other):
        if isinstance(other, (int, self.subclass)):
            return self.__class__([x*other for x in self.value])
        m1, m2, m3, m4 = self.value
        o1, o2, o3, o4 = self._to_list(other)
        o_LL = [m1*o1 - m2*o2, m1*o2 + m2*o1]
        o_LR = [m1*o3 - m2*o4, m1*o4 + m2*o3]
        o_RL = [m3*o1 - m4*o2, m3*o2 + m4*o1]
        o_RR = [m3*o3 - m4*o4, m3*o4 + m4*o3]
        o = [
            o_LL[0] - (o_RR[0] - o_RR[1]*self.extension_i),
            o_LL[1] - (o_RR[1] + o_RR[0]*self.extension_i),
            o_LR[0] + o_RL[0],
            o_LR[1] + o_RL[1]
        ]
        return self.__class__(o)

    __radd__ = __add__
    __rmul__ = __mul__

    def __pow__(self, other):
        if other == 0:
            return self.__class__([1,0,0,0])
        elif other == 1:
            return self
        elif other == 2:
            return self * self
        else:
            return self.__pow__(other % 2) * self.__pow__(other // 2) ** 2

    def inv(self):
        # return self ** (self.modulus ** 4 - 2)
        x0, x1, x2, x3 = self.value
        r20 = x2*x2 - x3*x3
        r21 = 2 * x2 * x3
        denom0 = x0**2 - x1**2 + r20 - r21 * 2
        denom1 = 2*x0*x1 + r21 + r20 * 2
        inv_denom_norm = (denom0 ** 2 + denom1 ** 2).inv()
        inv_denom0 = denom0 * inv_denom_norm
        inv_denom1 = -denom1 * inv_denom_norm
        o = self.__class__([
            x0 * inv_denom0 - x1 * inv_denom1,
            x0 * inv_denom1 + x1 * inv_denom0,
            -x2 * inv_denom0 + x3 * inv_denom1,
            -x2 * inv_denom1 - x3 * inv_denom0,
        ])
        return o
    
    def __truediv__(self, other):
        other = self.__class__(self._to_list(other))
        if other.value[1:] == [0,0,0]:
            factor = other.value[0].inv()
            return self.__class__([v * factor for v in self.value])
        else:
            return self * other.inv()

    def __eq__(self, other):
        return self.value == self._to_list(other)

    def __repr__(self):
        return '<'+str([v.value for v in self.value])+'>'

    def to_bytes(self):
        return b''.join([v.to_bytes() for v in self.value])

    @classmethod
    def from_bytes(cls, bytez):
        return cls([
            int.from_bytes(bytez[i:i+4], 'little') for i in range(0, 16, 4)
        ])

class ExtendedBabyMersenneElement(ExtendedFieldElement):
    subclass = BabyMersenneElement
    extension_i = 4
EBB = ExtendedBabyMersenneElement

In [6]:
EBB([1,2,3,4]) ** (31**4-1)

<[1, 0, 0, 0]>

In [7]:
EBB([1,2,3,4])*EBB([5,6,7,8])

<[2, 1, 3, 4]>

## Circle Group 

Adding two points on the unit circle.
$$
\begin{aligned}
& x_{\text {new }}=x 1 \cdot x 2-y 1 \cdot y 2=\cos \theta_1 \cdot \cos \theta_2-\sin \theta_1 \cdot \sin \theta_2=\cos \left(\theta_1+\theta_2\right) \\
& y_{\text {new }}=x 1 \cdot y 2+x 2 \cdot y 1=\cos \theta_1 \cdot \sin \theta_2+\cos \theta_2 \cdot \sin \theta_1=\sin \left(\theta_1+\theta_2\right)
\end{aligned}
$$

In [6]:
def point_add(pt1, pt2):
    (x1, y1), (x2, y2) = pt1, pt2
    return (
        x1 * x2 - y1 * y2,
        x1 * y2 + x2 * y1
    )

Doubling a point on the unit circle. This operation corresponds to doubling the angle $\theta$ of a point $(\cos \theta, \sin \theta)$ on the unit circle. The formulas used in the function align with the double-angle formulas for cosine and sine:
- $\cos (2 \theta)=2 \cos ^2(\theta)-1$
- $\sin (2 \theta)=2 \sin (\theta) \cos (\theta)$

In [8]:
def point_double(pt):
    x1, y1 = pt
    return (2 * x1 * x1 - 1, 2 * x1 * y1)

![](./circlestark.png)

## References
- https://github.com/Plonky3/Plonky3/tree/main/mersenne-31
- https://eprint.iacr.org/2024/278.pdf
- [Circle STARKs: Part I, Mersenne](https://www.zksecurity.xyz/blog/posts/circle-starks-1/)
- [Ariel Gabizon - FFT's on the projective line and circle-STARKs](https://www.youtube.com/watch?v=d1f9sBajj10)
- [Vitalik: Exploring circle STARKs](https://vitalik.eth.limo/general/2024/07/23/circlestarks.html)
- [Finite field - Wikipedia](https://en.wikipedia.org/wiki/Finite_field)
- [Circle group](https://en.wikipedia.org/wiki/Circle_group)